In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from PIL import Image

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling1D, AveragePooling2D, Dense, Dropout, Flatten, BatchNormalization, Input, Activation, Add

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

import sys
print("Python Version:", sys.version)
print("TensorFlow Version", tf.__version__)

Python Version: 3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
TensorFlow Version 2.16.1


In [3]:
# using GPU for faster processing

phys_devices = tf.config.list_physical_devices('GPU')
if len(phys_devices) > 0:
    # use the first visible GPU
    tf.config.experimental.set_visible_devices(phys_devices[0], 'GPU')
    print(f"Using GPU device {phys_devices[0].name}\n")
else:
    print("No GPU devices found!\n")

if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

No GPU devices found!

GPU is not available


In [4]:
# load furniture data from CSV files
furniture_df = pd.read_csv("furniture_data.csv")
furniture_sliding_df = pd.read_csv("furniture_sliding_data.csv")

# combine both datasets into a single dataframe
full_furniture_df = pd.concat([furniture_df, furniture_sliding_df], axis=0)

In [5]:
dev, test = train_test_split(furniture_df, train_size = 0.85, shuffle = True, random_state = 333)

In [6]:
dev.shape, test.shape

((76571, 3), (13513, 3))

In [7]:
# define batch size and image dimensions
BATCH_SIZE = 32
IMG_WIDTH  = 224
IMG_HEIGHT = 224

# data augmentation for training
train_datagen = ImageDataGenerator(rescale = 1/255.0,
                                width_shift_range = 0.05,
                                height_shift_range = 0.05,
                                rotation_range = 40,
                                zoom_range = 0.1,
                                shear_range = 0.2,
                                brightness_range = [0.5, 1.0],
                                validation_split = 0.15, 
                                fill_mode = 'nearest',
                                horizontal_flip = True,)

# data normalization for testing
test_datagen = ImageDataGenerator(rescale = 1/255.0)

In [8]:
# create data generators for training, validation, and testing
train_ds = train_datagen.flow_from_dataframe(
    dataframe = dev,
    x_col = 'Image_Path',
    y_col = 'Furniture_Category',
    color_mode = 'rgb',
    class_mode = 'categorical',
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = 42,
    subset = 'training'
)

val_ds = train_datagen.flow_from_dataframe(
    dataframe = dev,
    x_col = 'Image_Path',
    y_col = 'Furniture_Category',
    color_mode = 'rgb',
    class_mode = 'categorical',
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = 42,
    subset = 'validation'
)

test_ds = test_datagen.flow_from_dataframe(
    dataframe = test,
    x_col = 'Image_Path',
    y_col = 'Furniture_Category',
    color_mode = 'rgb',
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    class_mode = 'categorical',
    batch_size = BATCH_SIZE,
    shuffle = False  # maintain order for testing
)

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


In [9]:
print(train_ds.class_indices)

{}


In [10]:
# Check image
batchX, batchY = train_ds.__next__()

print(batchX.shape)
print(batchY.shape)

for i in range(3):
    image = batchX[i]
    label = batchY[i]

    print('Label: ', label)

    plt.imshow(image)
    plt.show()

(0, 224, 224, 3)
(0, 0)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import tensorflow as tf

def identity_block(x, filters):
    x_skip = x
    
    # first layer
    x = tf.keras.layers.Conv2D(filters[0], kernel_size = (1, 1), strides = (1, 1), padding = 'valid')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # second layer
    x = tf.keras.layers.Conv2D(filters[1], kernel_size = (3, 3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # third layer
    x = tf.keras.layers.Conv2D(filters[2], kernel_size = (1, 1), strides = (1, 1), padding = 'valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # input tensor (identity mapping)
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)

    return x

def convolutional_block(x, filters, strides = (2, 2)):
    x_skip = x

    # first convolutional layer
    x = tf.keras.layers.Conv2D(filters[0], kernel_size = (1, 1), strides = strides, padding = 'valid')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # second convolutional layer
    x = tf.keras.layers.Conv2D(filters[1], kernel_size = (3, 3), strides = (1, 1), padding = 'same')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # third convolutional layer
    x = tf.keras.layers.Conv2D(filters[2], kernel_size = (1, 1), strides = (1, 1), padding = 'valid')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # projection shortcut
    x_skip = tf.keras.layers.Conv2D(filters[2], kernel_size = (1, 1), strides = strides, padding = 'valid')(x_skip)
    x_skip = tf.keras.layers.BatchNormalization()(x_skip)

    # input tensor and projection shortcut
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)

    return x

def ResNet(shape = (224, 224, 3), classes = 1000, stages = [3, 4, 6, 3], model_name = "ResNet"):
    # input layer
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D(padding = (3, 3))(x_input)

    # stage 1
    x = tf.keras.layers.Conv2D(64, kernel_size = (7, 7), strides = (2, 2), padding = 'valid')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')(x)

    # stage 2
    x = convolutional_block(x, filters = [64, 64, 256], strides = (1, 1))
    for _ in range(stages[0] - 1):
        x = identity_block(x, filters = [64, 64, 256])

    # stage 3
    x = convolutional_block(x, filters = [128, 128, 512])
    for _ in range(stages[1] - 1):
        x = identity_block(x, filters = [128, 128, 512])

    # stage 4
    x = convolutional_block(x, filters = [256, 256, 1024])
    for _ in range(stages[2] - 1):
        x = identity_block(x, filters = [256, 256, 1024])

    # stage 5
    x = convolutional_block(x, filters = [512, 512, 2048])
    for _ in range(stages[3] - 1):
        x = identity_block(x, filters = [512, 512, 2048])

    # output layer
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)

    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = model_name)
    return model


def ResNet152(shape, classes):
    return ResNet(shape, classes, stages=[3, 8, 36, 3], model_name="ResNet152")


# create ResNet152 model
resnet152_model = ResNet152(shape = (224, 224, 3), classes = 102)
resnet152_model.summary()


In [ ]:
# get unique furniture categories from the dataframe
cat_folders = furniture_df["Furniture_Category"].unique()
print(cat_folders)

# calculate the total number of unique categories
num_classes = furniture_df["Furniture_Category"].nunique()
print("Number of categories:", num_classes)

In [ ]:
# custom CNN model

model = Sequential([
    # first convolutional block
    # input layer: 32 filters, 3x3 kernel size, 'same' padding, ReLU activation
    Conv2D(32, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)),
    MaxPooling2D(pool_size = (2,2), strides = (2,2)),

    # second convolutional block
    Conv2D(64, kernel_size = (3,3), padding = 'same', activation = 'relu'),
    MaxPooling2D(pool_size = (2,2), strides = (2,2)),

    # third convolutional block
    Conv2D(64, kernel_size = (3,3), padding = 'same', activation = 'relu'),
    MaxPooling2D(pool_size = (2,2), strides = (2,2)),

    Flatten(),
    Dropout(0.5),
    Dense(units = 64, activation = 'relu'),

    # output layer
    Dense(num_classes, activation = 'softmax')
])

model.summary()

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import CategoricalAccuracy, BinaryAccuracy
from keras.losses import CategoricalCrossentropy, BinaryCrossentropy

# Adam configuration
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback configuration
early_stopping_callback = EarlyStopping(
    monitor = 'val_accuracy',  # monitor validation accuracy
    patience = 15,  # stop if no improvement after 10 epochs
    restore_best_weights = True,   # restore weights with best performance
    verbose = 1,
    min_delta = 0.0001,
)

# Train the model
history = model.fit(train_ds, validation_data = val_ds, epochs = 5, batch_size = BATCH_SIZE, callbacks = [early_stopping_callback])

In [ ]:
# line Chart for showing loss over training

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.legend(['train', 'val'], loc = 'upper left')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

# line chart for showing accuracy over training

plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()

In [ ]:
# Test final accuracy
batchXTest, batchYTest = val_ds.__next__()

score = model.evaluate(batchXTest, batchYTest)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes = True, show_layer_names = True)

In [ ]:
model.save('../TrainingModels/furniture_cat_classification_model.h5')